In [1]:
# Testing the addition of NCMC moves in saltwap.
# Importing:
from datetime import datetime
from simtk import openmm, unit
from simtk.openmm import app
import numpy as np
import sys
sys.path.append("../saltswap/")
import saltswap

# Defining constants.
kB = unit.BOLTZMANN_CONSTANT_kB * unit.AVOGADRO_CONSTANT_NA
pressure = 1*unit.atmospheres
temperature = 300*unit.kelvin
delta_chem = -100*unit.kilojoule_per_mole

In [2]:
#Loading a premade water box:
pdb = app.PDBFile('../examples/waterbox.pdb')
forcefield = app.ForceField('tip3p.xml')
system = forcefield.createSystem(pdb.topology,nonbondedMethod=app.PME, nonbondedCutoff=1.0*unit.nanometer, constraints=app.HBonds)

In [3]:
# Minimizing energy with a temporary integrator.
integrator = openmm.VerletIntegrator(1.0 * unit.femtoseconds)
context = openmm.Context(system, integrator)
context.setPositions(pdb.positions)
openmm.LocalEnergyMinimizer.minimize(context, 1.0, 25)
positions = context.getState(getPositions=True).getPositions(asNumpy=True)
del context, integrator

In [4]:
# Defining the new system integrator that will be used for the standard MD moves:
integrator = openmm.LangevinIntegrator(temperature, 1/unit.picosecond, 0.002*unit.picoseconds)
system.addForce(openmm.MonteCarloBarostat(pressure, temperature, 25))
# Initialising the constant salt class. Inside, a 'compoundintegrator' is defined to do NCMC
#     NCMC parameters:
nkernals = 5           # Number of NCMC steps
nverlet = 5            # Number of velocity verlet steps in each NCMC step     
mc_saltswap = saltswap.SaltSwap(system=system,topology=pdb.topology,temperature=temperature,delta_chem=delta_chem,
                                integrator=integrator,pressure=pressure,
                                debug=False,nkernals=nkernals, nverlet_steps=nverlet)

In [5]:
# Creating the working the context
platform = openmm.Platform.getPlatformByName('CPU')
context = openmm.Context(system, mc_saltswap.compound_integrator,platform)
context.setPositions(positions)

In [6]:
# Brief thermalisation:
context.setVelocitiesToTemperature(temperature)
integrator.step(10)
positions = context.getState(getPositions=True,enforcePeriodicBox=True).getPositions(asNumpy=True)

In [7]:
# Cycles of MD and MC salt exchanges:
iterations = 100         # Number of rounds of MD and constant salt moves
nsteps = 5           # Amount of MD steps per iteration. 250000 steps = 500 picoseconds
nattempts = 5        # Number of identity exchanges attempts per iteration 
startTime = datetime.now()
for i in range(iterations):
    integrator.step(nsteps)
    mc_saltswap.update(context,nattempts=nattempts)
    print "Step %i complete" % i
tm = datetime.now() - startTime

print "Simulation time = ",tm.seconds
print "Acceptance probability =",mc_saltswap.getAcceptanceProbability()

../saltswap/saltswap.py:478: VisibleDeprecationWarning: converting an array with ndim > 0 to an index will result in an error in the future
  molecule = [atom for atom in self.mutable_residues[exchange_indices[0]].atoms()]
../saltswap/saltswap.py:489: VisibleDeprecationWarning: converting an array with ndim > 0 to an index will result in an error in the future
  molecule = [atom for atom in self.mutable_residues[exchange_indices[1]].atoms()]
../saltswap/saltswap.py:429: VisibleDeprecationWarning: converting an array with ndim > 0 to an index will result in an error in the future
  self.mutable_residues[exchange_indices[0]].name = self.waterName
../saltswap/saltswap.py:430: VisibleDeprecationWarning: converting an array with ndim > 0 to an index will result in an error in the future
  self.mutable_residues[exchange_indices[1]].name = self.waterName


Step 0 complete


../saltswap/saltswap.py:456: VisibleDeprecationWarning: converting an array with ndim > 0 to an index will result in an error in the future
  molecule = [atom for atom in self.mutable_residues[exchange_indices[0]].atoms()]
../saltswap/saltswap.py:467: VisibleDeprecationWarning: converting an array with ndim > 0 to an index will result in an error in the future
  molecule = [atom for atom in self.mutable_residues[exchange_indices[1]].atoms()]


Step 1 complete
Step 2 complete
Step 3 complete
Step 4 complete
Step 5 complete
Step 6 complete
Step 7 complete
Step 8 complete
Step 9 complete
Step 10 complete
Step 11 complete
Step 12 complete
Step 13 complete
Step 14 complete
Step 15 complete
Step 16 complete
Step 17 complete
Step 18 complete
Step 19 complete
Step 20 complete
Step 21 complete
Step 22 complete
Step 23 complete
Step 24 complete
Step 25 complete
Step 26 complete
Step 27 complete
Step 28 complete
Step 29 complete
Step 30 complete
Step 31 complete
Step 32 complete
Step 33 complete
Step 34 complete
Step 35 complete
Step 36 complete
Step 37 complete
Step 38 complete
Step 39 complete
Step 40 complete
Step 41 complete
Step 42 complete
Step 43 complete
Step 44 complete
Step 45 complete
Step 46 complete
Step 47 complete
Step 48 complete
Step 49 complete
Step 50 complete
Step 51 complete
Step 52 complete
Step 53 complete
Step 54 complete
Step 55 complete
Step 56 complete
Step 57 complete
Step 58 complete
Step 59 complete
Step 6

With iterations =100, nsteps = 10, nattampts =5, Simulation time =  13:
- No NCMC:
    Simulation time =  7
    Acceptance probability = 0.016
- nkernals = 2, nverlet = 1:
    Simulation time = 24
    Acceptance probability = 0.09
    - repeat:
        Simulation time = 23
        Acceptance probability = 0.04    
- nkernals = 5, nverlet = 1:
   Simulation blows up?
- nkernals = 5, nverlet = 2:   
    Simulation time = 74
    Acceptance probability = 0.63
    - repeat:
        Simulation time = 74
        Acceptance probability = 0.682
- nkernals = 2, nverlet = 2
    Simulation blows up?
- nkernals = 10, nverlet = 2
    Simulation time =  137
    Acceptance probability = 0.07
    - repeat:
        Simulation time =  141
        Acceptance probability = 0.062
- nkernals = 5, nverlet = 5
    Simulation time =  166
    Acceptance probability = 0.06